### Multi-task Learning
(Developed from dacon_etri_base_mod3_2.ipynb)
* 전처리 X


* GridSearch 대신 Optuna 사용 (병렬화 용이)


In [1]:
from datetime import datetime

# 현재 날짜 및 시간 가져오기
now = datetime.now()
timestamp = now.strftime("%m%d_%H%M")  # 예: 0517_1530

# kmjng
submission_folder = '/home/user/miniconda3/envs/gpu_conda/src/lifelog-sleep-ictc-2025/minjeong/submission/'

# # kmjng-nb
# submission_folder = 'C:/Users/minjeong/Documents/my_env/env1/src/lifelog-sleep-ictc-2025/minjeong/submission'

submission_file = f'submission_final_mod5_{timestamp}.csv'

# kmjng
base_folder =  '/home/user/miniconda3/envs/gpu_conda/src/data/ETRI_lifelog_dataset'

# # kmjng-nb
# base_folder = 'C:/Users/minjeong/Documents/my_env/env1/src/data/ETRI_lifelog_dataset'

folder = '/ch2025_data_items'

data_dir = base_folder + folder 




In [2]:

import numpy as np 
import pandas as pd 
import glob 
import random 
import os 
import matplotlib.pyplot as plt 
import seaborn as sns 
import ast 
import warnings
warnings.filterwarnings('ignore') 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import torch
import tensorflow as tf 


2025-06-03 22:59:20.809507: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-03 22:59:20.814732: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-03 22:59:20.897716: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-03 22:59:20.900037: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-03 22:59:21.760299: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
# seed 고정 
SD = 42 
random.seed(SD) 
np.random.seed(SD) 
os.environ['PYTHONHASHSEED'] = str(SD)
tf.random.set_seed(SD)  # TensorFlow 시드 설정

# 파일 경로 설정 - VSCode 상대경로로 변경
# 실제 경로에 맞게 수정 필요


# Parquet 파일 전체 경로 리스트 
parquet_files = glob.glob(os.path.join(data_dir, 'ch2025_*.parquet')) 

In [4]:
# 파일 이름을 키로, DataFrame을 값으로 저장할 딕셔너리 
lifelog_data = {} 

# 파일별로 읽기 
for file_path in parquet_files: 
    name = os.path.basename(file_path).replace('.parquet', '').replace('ch2025_', '') 
    lifelog_data[name] = pd.read_parquet(file_path) 
    print(f"✅ Loaded: {name}, shape = {lifelog_data[name].shape}") 

# 딕셔너리에 있는 모든 항목을 독립적인 변수로 할당 
for key, df in lifelog_data.items(): 
    globals()[f"{key}_df"] = df 

✅ Loaded: wHr, shape = (382918, 3)
✅ Loaded: mGps, shape = (800611, 3)
✅ Loaded: mScreenStatus, shape = (939653, 3)
✅ Loaded: mACStatus, shape = (939896, 3)
✅ Loaded: mActivity, shape = (961062, 3)
✅ Loaded: mBle, shape = (21830, 3)
✅ Loaded: mWifi, shape = (76336, 3)
✅ Loaded: mLight, shape = (96258, 3)
✅ Loaded: mAmbience, shape = (476577, 3)
✅ Loaded: wLight, shape = (633741, 3)
✅ Loaded: wPedo, shape = (748100, 9)
✅ Loaded: mUsageStats, shape = (45197, 3)


In [5]:
# 메트릭스 파일 읽기
metrics_train = pd.read_csv(base_folder + '/ch2025_metrics_train.csv')
sample_submission = pd.read_csv(base_folder+'/ch2025_submission_sample.csv')

# ✅ 기준 쌍 (subject_id, lifelog_date) 
sample_submission['lifelog_date'] = pd.to_datetime(sample_submission['lifelog_date']) 
test_keys = set(zip(sample_submission['subject_id'], sample_submission['lifelog_date'].dt.date)) 

# ✅ DataFrame 별 timestamp 컬럼 수동 지정 
dataframes = { 
    'mACStatus': (mACStatus_df, 'timestamp'), 
    'mActivity': (mActivity_df, 'timestamp'), 
    'mAmbience': (mAmbience_df, 'timestamp'), 
    'mBle': (mBle_df, 'timestamp'), 
    'mGps': (mGps_df, 'timestamp'), 
    'mLight': (mLight_df, 'timestamp'), 
    'mScreenStatus': (mScreenStatus_df, 'timestamp'), 
    'mUsageStats': (mUsageStats_df, 'timestamp'), 
    'mWifi': (mWifi_df, 'timestamp'), 
    'wHr': (wHr_df, 'timestamp'), 
    'wLight': (wLight_df, 'timestamp'), 
    'wPedo': (wPedo_df, 'timestamp'), 
} 

In [6]:
# ✅ 분리 함수 
def split_test_train(df, subject_col='subject_id', timestamp_col='timestamp'): 
    df[timestamp_col] = pd.to_datetime(df[timestamp_col], errors='coerce') 
    df = df.dropna(subset=[timestamp_col]) 
    df['date_only'] = df[timestamp_col].dt.date 
    df['key'] = list(zip(df[subject_col], df['date_only'])) 
    test_df = df[df['key'].isin(test_keys)].drop(columns=['date_only', 'key']) 
    train_df = df[~df['key'].isin(test_keys)].drop(columns=['date_only', 'key']) 
    return test_df, train_df 


In [7]:
# ✅ 결과 저장 
for name, (df, ts_col) in dataframes.items(): 
    print(f"⏳ {name} 분리 중...") 
    test_df, train_df = split_test_train(df.copy(), subject_col='subject_id', timestamp_col=ts_col) 
    globals()[f"{name}_test"] = test_df 
    globals()[f"{name}_train"] = train_df 
    print(f"✅ {name}_test → {test_df.shape}, {name}_train → {train_df.shape}") 

⏳ mACStatus 분리 중...
✅ mACStatus_test → (335849, 3), mACStatus_train → (604047, 3)
⏳ mActivity 분리 중...
✅ mActivity_test → (343579, 3), mActivity_train → (617483, 3)
⏳ mAmbience 분리 중...
✅ mAmbience_test → (170453, 3), mAmbience_train → (306124, 3)
⏳ mBle 분리 중...
✅ mBle_test → (8140, 3), mBle_train → (13690, 3)
⏳ mGps 분리 중...
✅ mGps_test → (287386, 3), mGps_train → (513225, 3)
⏳ mLight 분리 중...
✅ mLight_test → (34439, 3), mLight_train → (61819, 3)
⏳ mScreenStatus 분리 중...
✅ mScreenStatus_test → (336160, 3), mScreenStatus_train → (603493, 3)
⏳ mUsageStats 분리 중...
✅ mUsageStats_test → (16499, 3), mUsageStats_train → (28698, 3)
⏳ mWifi 분리 중...
✅ mWifi_test → (27467, 3), mWifi_train → (48869, 3)
⏳ wHr 분리 중...
✅ wHr_test → (143311, 3), wHr_train → (239607, 3)
⏳ wLight 분리 중...
✅ wLight_test → (233809, 3), wLight_train → (399932, 3)
⏳ wPedo 분리 중...
✅ wPedo_test → (288832, 9), wPedo_train → (459268, 9)


In [8]:
def process_mACStatus(df): 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    df = df.sort_values(['subject_id', 'timestamp']) 
    results = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        status = group['m_charging'].values # 0/1 상태 
        times = group['timestamp'].values # 충전 상태 비율 
        ratio_charging = status.mean() 
        # 상태 전이 횟수 
        transitions = (status[1:] != status[:-1]).sum() 
        # 연속된 1 상태 길이들 
        lengths = [] 
        current_len = 0 
        for val in status: 
            if val == 1: 
                current_len += 1 
            elif current_len > 0: 
                lengths.append(current_len) 
                current_len = 0 
        if current_len > 0: 
            lengths.append(current_len) 
        avg_charging_duration = np.mean(lengths) if lengths else 0 
        max_charging_duration = np.max(lengths) if lengths else 0 
        results.append({ 
            'subject_id': subj, 
            'date': date, 
            'charging_ratio': ratio_charging, 
            'charging_transitions': transitions, 
            'avg_charging_duration': avg_charging_duration, 
            'max_charging_duration': max_charging_duration, 
        }) 
    return pd.DataFrame(results) 

mACStatus_df2 = process_mACStatus(mACStatus_df) 

In [9]:
def process_mActivity(df): 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    summary = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        counts = group['m_activity'].value_counts(normalize=True) # 비율 
        row = {'subject_id': subj, 'date': date} 
        # 0~8 비율 저장 
        for i in range(9): 
            row[f'activity_{i}_ratio'] = counts.get(i, 0) 
        # 주요 활동 정보 
        row['dominant_activity'] = group['m_activity'].mode()[0] 
        row['num_unique_activities'] = group['m_activity'].nunique() 
        summary.append(row) 
    return pd.DataFrame(summary) 

mActivity_df2 = process_mActivity(mActivity_df) 

In [10]:
mActivity_df2['activity_2_ratio'].value_counts(ascending=False)

activity_2_ratio
0    700
Name: count, dtype: int64

In [11]:
mActivity_df2.drop(columns=['activity_2_ratio', 'activity_5_ratio','activity_6_ratio'], axis=1, inplace=True) # 2번('bicycle'), 5번('tilting'), 6번 모두 0

In [12]:
mActivity_df2.head(3)

,subject_id,date,activity_0_ratio,activity_1_ratio,activity_3_ratio,activity_4_ratio,activity_7_ratio,activity_8_ratio,dominant_activity,num_unique_activities
0,id01,2024-06-26,0.125176,0.001406,0.672293,0.157525,0.043601,0.0,3,5
1,id01,2024-06-27,0.146528,0.000000,0.611111,0.220833,0.021528,0.0,3,4
2,id01,2024-06-28,0.111806,0.000694,0.861806,0.000694,0.025000,0.0,3,5


In [13]:
# 지정된 10개 라벨 
top_10_labels = [ 
    "Inside, small room", "Speech", "Silence", "Music", "Narration, monologue", 
    "Child speech, kid speaking", "Conversation", "Speech synthesizer", "Shout", "Babbling" 
] 

def process_mAmbience_top10(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    # 초기화 
    for label in top_10_labels + ['others']: 
        df[label] = 0.0 
    for idx, row in df.iterrows(): 
        parsed = ast.literal_eval(row['m_ambience']) if isinstance(row['m_ambience'], str) else row['m_ambience'] 
        others_prob = 0.0 
        for label, prob in parsed: 
            prob = float(prob) 
            if label in top_10_labels: 
                df.at[idx, label] = prob 
            else: 
                others_prob += prob 
        df.at[idx, 'others'] = others_prob 
    return df.drop(columns=['m_ambience']) 

mAmbience_df2= process_mAmbience_top10(mAmbience_df) 

In [14]:
def summarize_mAmbience_daily(df): 
    prob_cols = [col for col in df.columns if col not in ['subject_id', 'timestamp', 'date']] 
    # 하루 단위로 평균값 요약 
    daily_summary = df.groupby(['subject_id', 'date'])[prob_cols].mean().reset_index() 
    return daily_summary 

mAmbience_df2 = summarize_mAmbience_daily(mAmbience_df2) 


In [15]:
def process_mBle(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    features = [] 
    for idx, row in df.iterrows(): 
        entry = ast.literal_eval(row['m_ble']) if isinstance(row['m_ble'], str) else row['m_ble'] 
        rssi_list = [] 
        class_0_cnt = 0 
        class_other_cnt = 0 
        for device in entry: 
            try: 
                rssi = int(device['rssi']) 
                rssi_list.append(rssi) 
                if str(device['device_class']) == '0': 
                    class_0_cnt += 1 
                else: 
                    class_other_cnt += 1 
            except: 
                continue # malformed record 
        feature = { 
            'subject_id': row['subject_id'], 
            'date': row['date'], 
            'device_class_0_cnt': class_0_cnt, 
            'device_class_others_cnt': class_other_cnt, 
            'device_count': len(rssi_list), 
            'rssi_mean': np.mean(rssi_list) if rssi_list else np.nan, 
            'rssi_min': np.min(rssi_list) if rssi_list else np.nan, 
            'rssi_max': np.max(rssi_list) if rssi_list else np.nan, 
        } 
        features.append(feature) 
    return pd.DataFrame(features) 

In [16]:
def summarize_mBle_daily(df): 
    # row 단위 BLE feature 추출 
    df = process_mBle(df) 
    # 하루 단위로 cnt 합치기 
    grouped = df.groupby(['subject_id', 'date']).agg({ 
        'device_class_0_cnt': 'sum', 
        'device_class_others_cnt': 'sum', 
        'rssi_mean': 'mean', 
        'rssi_min': 'min', 
        'rssi_max': 'max', 
    }).reset_index() 
    # 총합 구해서 비율 계산 
    total_cnt = grouped['device_class_0_cnt'] + grouped['device_class_others_cnt'] 
    grouped['device_class_0_ratio'] = grouped['device_class_0_cnt'] / total_cnt.replace(0, np.nan) 
    grouped['device_class_others_ratio'] = grouped['device_class_others_cnt'] / total_cnt.replace(0, np.nan) 
    # 필요 없는 원래 cnt 컬럼 제거 
    grouped.drop(columns=['device_class_0_cnt', 'device_class_others_cnt'], inplace=True) 
    return grouped 

mBle_df2 = summarize_mBle_daily(mBle_df) 

In [17]:
def process_mGps(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    features = [] 
    for idx, row in df.iterrows(): 
        gps_list = ast.literal_eval(row['m_gps']) if isinstance(row['m_gps'], str) else row['m_gps'] 
        altitudes = [] 
        latitudes = [] 
        longitudes = [] 
        speeds = [] 
        for entry in gps_list: 
            try: 
                altitudes.append(float(entry['altitude'])) 
                latitudes.append(float(entry['latitude'])) 
                longitudes.append(float(entry['longitude'])) 
                speeds.append(float(entry['speed'])) 
            except: 
                continue 
        features.append({ 
            'subject_id': row['subject_id'], 
            'date': row['date'], 
            'altitude_mean': np.mean(altitudes) if altitudes else np.nan, 
            'latitude_std': np.std(latitudes) if latitudes else np.nan, 
            'longitude_std': np.std(longitudes) if longitudes else np.nan, 
            'speed_mean': np.mean(speeds) if speeds else np.nan, 
            'speed_max': np.max(speeds) if speeds else np.nan, 
            'speed_std': np.std(speeds) if speeds else np.nan, 
        }) 
    return pd.DataFrame(features) 

In [18]:
m_Gps_df2 = process_mGps(mGps_df) 
m_Gps_df2 = m_Gps_df2.groupby(['subject_id', 'date']).agg({ 
    'altitude_mean': 'mean', 
    'latitude_std': 'mean', 
    'longitude_std': 'mean', 
    'speed_mean': 'mean', 
    'speed_max': 'max', 
    'speed_std': 'mean' 
}).reset_index() 

In [19]:
def process_mLight(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    df['hour'] = df['timestamp'].dt.hour 
    # 밤(22~05시), 낮(06~21시) 구분 
    df['is_night'] = df['hour'].apply(lambda h: h >= 22 or h < 6) 
    # 하루 단위 요약 
    daily = df.groupby(['subject_id', 'date']).agg( 
        light_mean=('m_light', 'mean'), 
        light_std=('m_light', 'std'), 
        light_max=('m_light', 'max'), 
        light_min=('m_light', 'min'), 
        light_night_mean=('m_light', lambda x: x[df.loc[x.index, 'is_night']].mean()), 
        light_day_mean=('m_light', lambda x: x[~df.loc[x.index, 'is_night']].mean()), 
        light_night_ratio=('is_night', 'mean') # 밤 시간 측정 비율 
    ).reset_index() 
    return daily 

mLight_df2 = process_mLight(mLight_df) 

In [20]:
def process_mScreenStatus(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    features = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        status = group['m_screen_use'].values 
        ratio_on = status.mean() 
        transitions = (status[1:] != status[:-1]).sum() 
        # 연속된 1 상태 길이들 
        durations = [] 
        current = 0 
        for val in status: 
            if val == 1: 
                current += 1 
            elif current > 0: 
                durations.append(current) 
                current = 0 
        if current > 0: 
            durations.append(current) 
        features.append({ 
            'subject_id': subj, 
            'date': date, 
            'screen_on_ratio': ratio_on, 
            'screen_on_transitions': transitions, 
            'screen_on_duration_avg': np.mean(durations) if durations else 0, 
            'screen_on_duration_max': np.max(durations) if durations else 0, 
        }) 
    return pd.DataFrame(features) 

mScreenStatus_df2 = process_mScreenStatus(mScreenStatus_df) 


In [21]:
top_apps = [ 
    'One UI 홈', '카카오톡', '시스템 UI', 'NAVER', '캐시워크', 
    '성경일독Q', 'YouTube', '통화', '메시지', '타임스프레드', 'Instagram'
] 

def process_mUsageStats(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    features = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        app_time = {app: 0 for app in top_apps} 
        others_time = 0 
        for row in group['m_usage_stats']: 
            parsed = ast.literal_eval(row) if isinstance(row, str) else row 
            for entry in parsed: 
                app = entry.get('app_name') 
                time = entry.get('total_time', 0) 
                if app in top_apps: 
                    app_time[app] += int(time) 
                else: 
                    others_time += int(time) 
        feature = { 
            'subject_id': subj, 
            'date': date, 
            'others_time': others_time 
        } 
        # 각 앱별 컬럼 추가 
        feature.update({f'{app}_time': app_time[app] for app in top_apps}) 
        features.append(feature) 
    return pd.DataFrame(features) 

mUsageStats_df2 = process_mUsageStats(mUsageStats_df) 

In [22]:

def process_mWifi(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    results = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        rssi_all = [] 
        for row in group['m_wifi']: 
            parsed = ast.literal_eval(row) if isinstance(row, str) else row 
            for ap in parsed: 
                try: 
                    rssi = int(ap['rssi']) 
                    rssi_all.append(rssi) 
                except: 
                    continue 
        results.append({ 
            'subject_id': subj, 
            'date': date, 
            'wifi_rssi_mean': np.mean(rssi_all) if rssi_all else np.nan, 
            'wifi_rssi_min': np.min(rssi_all) if rssi_all else np.nan, 
            'wifi_rssi_max': np.max(rssi_all) if rssi_all else np.nan, 
            'wifi_detected_cnt': len(rssi_all) 
        }) 
    return pd.DataFrame(results) 

mWifi_df2 = process_mWifi(mWifi_df) 

In [23]:

def get_time_block(hour): 
    if 0 <= hour < 6: 
        return 'early_morning' 
    elif 6 <= hour < 12: 
        return 'morning' 
    elif 12 <= hour < 18: 
        return 'afternoon' 
    else: 
        return 'evening' 

def process_wHr_by_timeblock(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    df['block'] = df['timestamp'].dt.hour.map(get_time_block) 
    results = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        block_stats = {'subject_id': subj, 'date': date} 
        for block, block_group in group.groupby('block'): 
            hr_all = [] 
            for row in block_group['heart_rate']: 
                parsed = ast.literal_eval(row) if isinstance(row, str) else row 
                hr_all.extend([int(h) for h in parsed if h is not None]) 
            if not hr_all: 
                continue 
            above_100 = [hr for hr in hr_all if hr > 100] 
            block_stats[f'hr_{block}_mean'] = np.mean(hr_all) 
            block_stats[f'hr_{block}_std'] = np.std(hr_all) 
            block_stats[f'hr_{block}_max'] = np.max(hr_all) 
            block_stats[f'hr_{block}_min'] = np.min(hr_all) 
            block_stats[f'hr_{block}_above_100_ratio'] = len(above_100) / len(hr_all) 
        results.append(block_stats) 
    return pd.DataFrame(results) 

wHr_df2 = process_wHr_by_timeblock(wHr_df) 

In [24]:

def process_wLight_by_timeblock(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    df['block'] = df['timestamp'].dt.hour.map(get_time_block) 
    results = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        block_stats = {'subject_id': subj, 'date': date} 
        for block, block_group in group.groupby('block'): 
            lux = block_group['w_light'].dropna().values 
            if len(lux) == 0: 
                continue 
            block_stats[f'wlight_{block}_mean'] = np.mean(lux) 
            block_stats[f'wlight_{block}_std'] = np.std(lux) 
            block_stats[f'wlight_{block}_max'] = np.max(lux) 
            block_stats[f'wlight_{block}_min'] = np.min(lux) 
        results.append(block_stats) 
    return pd.DataFrame(results) 

wLight_df2 = process_wLight_by_timeblock(wLight_df)

In [25]:
def process_wPedo(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    summary = df.groupby(['subject_id', 'date']).agg({ 
        'step': 'sum', 
        'step_frequency': 'mean', 
        'distance': 'sum', 
        'speed': ['mean', 'max'], 
        'burned_calories': 'sum' 
    }).reset_index() 
    # 컬럼 이름 정리 
    summary.columns = ['subject_id', 'date', 'step_sum', 'step_frequency_mean', 'distance_sum', 'speed_mean', 'speed_max', 'burned_calories_sum'] 
    return summary 

wPedo_df2 = process_wPedo(wPedo_df) 

In [26]:

from functools import reduce 
df_list = [ 
    mACStatus_df2, 
    mActivity_df2,  
    mAmbience_df2,  
    mBle_df2, 
    m_Gps_df2, 
    mLight_df2, 
    mScreenStatus_df2, 
    mUsageStats_df2, 
    mWifi_df2, 
    wHr_df2, 
    wLight_df2, 
    wPedo_df2 
] 

merged_df = reduce(lambda left, right: pd.merge(left, right, on=['subject_id', 'date'], how='outer'), df_list) 


* train, test

In [27]:

# metrics_train의 lifelog_date → datetime.date 형으로 변환 
metrics_train['lifelog_date'] = pd.to_datetime(metrics_train['lifelog_date']).dt.date 

# merged_df의 date도 변환 
merged_df['date'] = pd.to_datetime(merged_df['date']).dt.date 

# 1. date 기준 정렬을 위해 metrics_train의 lifelog_date -> date로 맞추기 
metrics_train_renamed = metrics_train.rename(columns={'lifelog_date': 'date'}) 

# 2. train_df: metrics_train과 일치하는 (subject_id, date) → 라벨 포함 
train_df = pd.merge(metrics_train_renamed, merged_df, on=['subject_id', 'date'], how='inner') 

# 3. test_df: metrics_train에 없는 (subject_id, date) 
merged_keys = merged_df[['subject_id', 'date']] 
train_keys = metrics_train_renamed[['subject_id', 'date']] 
test_keys = pd.merge(merged_keys, train_keys, on=['subject_id', 'date'], how='left', indicator=True) 
test_keys = test_keys[test_keys['_merge'] == 'left_only'].drop(columns=['_merge']) 
test_df = pd.merge(test_keys, merged_df, on=['subject_id', 'date'], how='left') 


In [28]:

# ✅ 타겟 리스트 
targets_binary = ['Q1', 'Q2', 'Q3', 'S2', 'S3'] 
target_multiclass = 'S1' 

# ✅ feature 준비 
X = train_df.drop(columns=['subject_id', 'sleep_date', 'date', 'Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']) 
X.fillna(0, inplace=True) # 결측값 처리 

test_X = test_df.drop(columns=['subject_id', 'date']) 
test_X.fillna(0, inplace=True) 


In [29]:

# 컬럼 이름에서 특수 문자 제거/변환 
def sanitize_column_names(df): 
    df.columns = ( 
        df.columns 
        .str.replace(r"[^\w]", "_", regex=True) # 특수문자 → _ 
        .str.replace(r"__+", "_", regex=True) # 연속된 _ 제거 
        .str.strip("_") # 앞뒤 _ 제거 
    ) 
    return df 

# 모든 입력에 적용 
X = sanitize_column_names(X) 
test_X = sanitize_column_names(test_X) 

### 모델링 시작 

In [30]:
# 모든 특성에 대해 모델링 진행 
X_selected = X.copy()
test_X_selected = test_X.copy()

In [31]:
# LightGBM 라이브러리 임포트
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix

# GridSearch를 위한 교차 검증 설정
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# LightGBM 기본 파라미터 설정
lgbm_params = {
    'n_estimators': 1000,
    'learning_rate': 0.03,
    'random_state': 42,
    'n_jobs': -1,
    'verbosity': -1
}


### Feature 선정을 위한 학습

In [32]:
# -------------------------------- # 
# [Feature 선정 전 그리드서치 적용 X] 

# 각 모델의 feature importance를 numpy 배열로 저장해 쌓고, axis=0으로 평균을 냅니다.
# 평균값이 큰 순서대로 상위 20개 feature를 고릅니다.

import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier

# targets_binary: 이진 타겟 리스트, y_multi: 다중 타겟
all_targets = targets_binary + ['S1']  # S1이 다중 분류 타겟 예시
feature_names = X_selected.columns.tolist()
feature_importances = []

for col in all_targets:
    print(f">> {col} 변수 진행 중..")
    y = train_df[col]
    if col == 'S1':
        params = lgbm_params.copy()
        params['objective'] = 'multiclass'
        params['num_class'] = len(y.unique())
    else:
        params = lgbm_params.copy()
        params['objective'] = 'binary'
    model = LGBMClassifier(**params)
    model.fit(X_selected, y)
    feature_importances.append(model.feature_importances_)

# numpy 배열로 변환 후 평균
feature_importances = np.array(feature_importances)
mean_importance = feature_importances.mean(axis=0)

# 상위 20개 feature 추출
top20_idx = mean_importance.argsort()[::-1][:20]
top20_features = [feature_names[i] for i in top20_idx]
print("상위 20개 feature:", top20_features)

# 선택된 변수로 데이터셋 재구성
X_top20 = X_selected[top20_features]


>> Q1 변수 진행 중..
>> Q2 변수 진행 중..
>> Q3 변수 진행 중..
>> S2 변수 진행 중..
>> S3 변수 진행 중..
>> S1 변수 진행 중..
상위 20개 feature: ['light_night_mean', '카카오톡_time', 'others_time', '통화_time', '메시지_time', 'One_UI_홈_time', 'wifi_rssi_mean', 'light_max', 'activity_3_ratio', 'activity_7_ratio', 'screen_on_ratio', 'NAVER_time', 'screen_on_duration_avg', 'wifi_rssi_max', 'Music', 'avg_charging_duration', 'charging_ratio', 'rssi_mean', 'screen_on_duration_max', 'speed_max_x']


### Feature Importance (for binary, multi classification)

In [ ]:
# sudo apt update
# sudo apt install fonts-nanum
# sudo fc-cache -fv
# 나눔고딕 폰트 경로 설정 (한글인코딩)


* [★] 선정된 Feature 

In [33]:
top20_features

['light_night_mean',
 '카카오톡_time',
 'others_time',
 '통화_time',
 '메시지_time',
 'One_UI_홈_time',
 'wifi_rssi_mean',
 'light_max',
 'activity_3_ratio',
 'activity_7_ratio',
 'screen_on_ratio',
 'NAVER_time',
 'screen_on_duration_avg',
 'wifi_rssi_max',
 'Music',
 'avg_charging_duration',
 'charging_ratio',
 'rssi_mean',
 'screen_on_duration_max',
 'speed_max_x']

In [34]:
# 선택된 특성만으로 데이터셋 구성
X_selected = X[top20_features]
test_X_selected = test_X[top20_features]


### 오토인코더

In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

print("\n===== 오토인코더를 사용한 고급 특성 추출 =====")

# 하이퍼파라미터 설정
encoding_dim = min(64, X_selected.shape[1])  # 인코딩 차원 (원본 특성 수보다 작게 설정)
batch_size = 64
epochs = 100

# 입력 차원
input_dim = X_selected.shape[1]

# 오토인코더 모델 구축
input_layer = Input(shape=(input_dim,))

# 인코더 부분
encoded = Dense(128, activation='relu')(input_layer)
encoded = BatchNormalization()(encoded)
encoded = Dropout(0.3)(encoded)
encoded = Dense(64, activation='relu')(encoded)
encoded = BatchNormalization()(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# 디코더 부분
decoded = Dense(64, activation='relu')(encoded)
decoded = BatchNormalization()(decoded)
decoded = Dropout(0.3)(decoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = BatchNormalization()(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

# 전체 오토인코더 모델
autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# 인코더 부분만 추출 (특성 추출용)
encoder = Model(input_layer, encoded)



===== 오토인코더를 사용한 고급 특성 추출 =====


2025-06-03 23:03:02.961083: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:03:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-06-03 23:03:03.229568: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [36]:
# 훈련/검증 세트 분리
X_train_auto, X_val_auto = train_test_split(X_selected, test_size=0.2, random_state=42)

# 조기 종료 설정
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# 오토인코더 모델 학습
print("오토인코더 학습 중...")
autoencoder.fit(
    X_train_auto, X_train_auto,
    epochs=epochs,
    batch_size=batch_size,
    shuffle=True,
    validation_data=(X_val_auto, X_val_auto),
    callbacks=[early_stopping],
    verbose=1
)

# 인코딩된 특성 추출
print("인코딩된 특성 추출 중...")
encoded_features_train = encoder.predict(X_selected)
encoded_features_test = encoder.predict(test_X_selected)

# 원본 특성과 인코딩된 특성 결합
X_combined = np.hstack([X_selected, encoded_features_train])
test_X_combined = np.hstack([test_X_selected, encoded_features_test])

print(f"원본 특성 수: {X_selected.shape[1]}")
print(f"인코딩된 특성 수: {encoded_features_train.shape[1]}")
print(f"결합된 특성 수: {X_combined.shape[1]}")


오토인코더 학습 중...
Epoch 1/100
6/6 [==============================] - 1s 37ms/step - loss: 12277965127680.0000 - val_loss: 13506580578304.0000
Epoch 2/100
6/6 [==============================] - 0s 9ms/step - loss: 12277965127680.0000 - val_loss: 13506580578304.0000
Epoch 3/100
6/6 [==============================] - 0s 9ms/step - loss: 12277965127680.0000 - val_loss: 13506580578304.0000
Epoch 4/100
6/6 [==============================] - 0s 9ms/step - loss: 12277965127680.0000 - val_loss: 13506580578304.0000
Epoch 5/100
6/6 [==============================] - 0s 9ms/step - loss: 12277965127680.0000 - val_loss: 13506580578304.0000
Epoch 6/100
6/6 [==============================] - 0s 9ms/step - loss: 12277965127680.0000 - val_loss: 13506580578304.0000
Epoch 7/100
6/6 [==============================] - 0s 9ms/step - loss: 12277965127680.0000 - val_loss: 13506580578304.0000
Epoch 8/100
6/6 [==============================] - 0s 9ms/step - loss: 12277965127680.0000 - val_loss: 13506580578304.0000
E

### feature에 대한 모델링 

* 4_1 참고한 모델링

오토인코더를 통해 생성된 특성(X_combined, test_X_combined)을 사용하여  
LightGBM 모델을 Optuna로 튜닝하고 예측

In [37]:
# # 오토인코더 결과 결합 (필수X)
# X_combined = np.hstack([X_selected, encoded_features_train])  # 훈련 특성
# test_X_combined = np.hstack([test_X_selected, encoded_features_test])  # 테스트 특성


In [38]:
# ##################
# ## Optuna 
# ##################
# GPU 최적화된 LightGBM 하이퍼파라미터 튜닝

import optuna
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
import numpy as np
import time

# GPU 최적화 파라미터 설정
def get_optimized_gpu_params():
    """GPU 성능을 최대화하는 LightGBM 파라미터"""
    return {
        'device': 'gpu',
        'gpu_use_dp': True,  # 더블 정밀도 사용 (정확도 향상)
        'num_gpu': 1,  # 사용할 GPU 개수
        'random_state': 42,
        'verbosity': -1,
        'n_jobs': 1,  # GPU 사용시 반드시 1
        'force_row_wise': True,  # GPU에서 더 빠른 학습
        'histogram_pool_size': -1,  # 메모리 최적화
        'gpu_platform_id': 0,
        'gpu_device_id': 0
    }

# Optuna를 사용한 효율적인 하이퍼파라미터 최적화
def optimize_lgbm_with_optuna(X, y, objective_type='binary', n_trials=50):
    """
    Optuna를 사용한 효율적인 하이퍼파라미터 최적화
    GridSearch보다 3-5배 빠름
    """
    
    def objective(trial):
        # 하이퍼파라미터 범위 설정 (더 좁은 범위로 빠른 탐색)
        params = get_optimized_gpu_params()
        
        # 중요한 파라미터만 최적화
        params.update({
            'n_estimators': trial.suggest_int('n_estimators', 100, 500, step=50),
            'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.2, step=0.05),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'num_leaves': trial.suggest_int('num_leaves', 20, 100, step=10),
            'subsample': trial.suggest_float('subsample', 0.7, 1.0, step=0.1),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0, step=0.1),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0, step=0.1),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0, step=0.1),
        })
        
        if objective_type == 'binary':
            params['objective'] = 'binary'
            model = lgb.LGBMClassifier(**params)
            scoring = 'f1'
        else:
            params['objective'] = 'multiclass'
            params['num_class'] = len(np.unique(y))
            model = lgb.LGBMClassifier(**params)
            scoring = 'f1_macro'
        
        # 더 빠른 교차검증 (3-fold 사용)
        try:
            scores = cross_val_score(model, X, y, cv=3, scoring=scoring, n_jobs=1)
            return scores.mean()
        except Exception as e:
            print(f"Trial failed: {e}")
            return 0.0
    
    # Optuna 최적화 실행
    study = optuna.create_study(direction='maximize', 
                              sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
    
    return study.best_params

# 배치 예측으로 속도 향상
def batch_predict(model, X_test, batch_size=10000):
    """
    대용량 데이터를 배치로 나누어 예측하여 메모리 효율성 향상
    """
    n_samples = X_test.shape[0]
    predictions = []
    
    for i in range(0, n_samples, batch_size):
        batch_end = min(i + batch_size, n_samples)
        batch_pred = model.predict(X_test[i:batch_end])
        predictions.extend(batch_pred)
    
    return np.array(predictions)

# 메인 최적화 실행 함수
def run_optimized_training(X_selected, train_df, test_X_selected, targets_binary):
    """최적화된 학습 실행"""
    
    print("🚀 GPU 최적화된 LightGBM 학습 시작")
    start_time = time.time()
    
    # GPU 사용 가능 여부 먼저 확인
    if not check_gpu_availability():
        print("❌ GPU 사용 불가. CPU 모드로 전환합니다.")
        return run_cpu_fallback(X_selected, train_df, test_X_selected, targets_binary)
    
    binary_preds_optimized = {}
    binary_models_optimized = {}
    
    # 이진 분류 모델들 최적화
    for i, col in enumerate(targets_binary):
        print(f"\n📊 {col} 최적화 중... ({i+1}/{len(targets_binary)})")
        y = train_df[col]
        
        # Optuna로 빠른 하이퍼파라미터 최적화
        best_params = optimize_lgbm_with_optuna(X_selected, y, 'binary', n_trials=30)
        
        # 최적 파라미터로 최종 모델 학습
        final_params = get_optimized_gpu_params()
        final_params.update(best_params)
        
        model = lgb.LGBMClassifier(**final_params)
        model.fit(X_selected, y)
        
        # 배치 예측으로 메모리 효율성 향상
        predictions = batch_predict(model, test_X_selected)
        
        binary_preds_optimized[col] = predictions
        binary_models_optimized[col] = model
        
        print(f"✅ {col} 완료")
    
    # 다중 분류 모델 (S1) 최적화
    print(f"\n📊 S1 다중분류 최적화 중...")
    y_multi = train_df['S1']
    
    best_params_multi = optimize_lgbm_with_optuna(X_selected, y_multi, 'multiclass', n_trials=30)
    
    final_params_multi = get_optimized_gpu_params()
    final_params_multi.update(best_params_multi)
    final_params_multi['objective'] = 'multiclass'
    final_params_multi['num_class'] = len(y_multi.unique())
    
    multi_model = lgb.LGBMClassifier(**final_params_multi)
    multi_model.fit(X_selected, y_multi)
    
    multiclass_pred_optimized = batch_predict(multi_model, test_X_selected)
    
    total_time = time.time() - start_time
    print(f"\n🎉 전체 학습 완료! 소요시간: {total_time:.2f}초")
    
    return binary_preds_optimized, binary_models_optimized, multiclass_pred_optimized, multi_model

def check_gpu_availability():
    """GPU 사용 가능 여부 빠른 확인"""
    try:
        # 더 작은 테스트 데이터로 빠른 확인
        from sklearn.datasets import make_classification
        X_test, y_test = make_classification(n_samples=50, n_features=5, random_state=42)
        
        params = get_optimized_gpu_params()
        params['n_estimators'] = 5
        
        model = lgb.LGBMClassifier(**params)
        model.fit(X_test, y_test)
        
        print("✅ GPU 사용 가능 확인됨")
        return True
    except Exception as e:
        print(f"❌ GPU 사용 불가: {str(e)[:100]}...")
        return False

def run_cpu_fallback(X_selected, train_df, test_X_selected, targets_binary):
    """GPU 실패시 CPU 대체 실행"""
    print("🔄 CPU 모드로 최적화된 학습 실행")
    
    cpu_params = {
        'device': 'cpu',
        'n_jobs': -1,
        'random_state': 42,
        'verbosity': -1
    }
    
    # CPU용 간단한 최적화 (더 빠름)
    binary_preds_cpu = {}
    binary_models_cpu = {}
    
    for col in targets_binary:
        y = train_df[col]
        
        # CPU용 기본 파라미터로 빠른 학습
        model = lgb.LGBMClassifier(
            objective='binary',
            n_estimators=200,
            learning_rate=0.1,
            max_depth=6,
            **cpu_params
        )
        
        model.fit(X_selected, y)
        binary_preds_cpu[col] = model.predict(test_X_selected)
        binary_models_cpu[col] = model
    
    # 다중분류
    y_multi = train_df['S1']
    multi_model_cpu = lgb.LGBMClassifier(
        objective='multiclass',
        num_class=len(y_multi.unique()),
        n_estimators=200,
        learning_rate=0.1,
        max_depth=6,
        **cpu_params
    )
    
    multi_model_cpu.fit(X_selected, y_multi)
    multiclass_pred_cpu = multi_model_cpu.predict(test_X_selected)
    
    return binary_preds_cpu, binary_models_cpu, multiclass_pred_cpu, multi_model_cpu


In [39]:
X_selected = X_combined 
test_X_selected = test_X_combined


In [40]:
# 경고 무시 설정
import warnings
warnings.filterwarnings('ignore')

results = run_optimized_training(X_selected, train_df, test_X_selected, targets_binary)
binary_preds_optimized, binary_models_optimized, multiclass_pred_optimized, multi_model = results

print("🎯 최적화 완료! 결과를 확인하세요.")

[I 2025-06-03 23:03:11,516] A new study created in memory with name: no-name-fa3b0d91-46d3-4268-b9c1-dee9769ae1f7


🚀 GPU 최적화된 LightGBM 학습 시작
✅ GPU 사용 가능 확인됨

📊 Q1 최적화 중... (1/5)


Best trial: 0. Best value: 0.369582:   3%|▎         | 1/30 [00:03<01:39,  3.42s/it]

[I 2025-06-03 23:03:14,936] Trial 0 finished with value: 0.3695823045838303 and parameters: {'n_estimators': 250, 'learning_rate': 0.2, 'max_depth': 8, 'num_leaves': 70, 'subsample': 0.7, 'colsample_bytree': 0.7, 'reg_alpha': 0.0, 'reg_lambda': 0.9}. Best is trial 0 with value: 0.3695823045838303.


Best trial: 1. Best value: 0.391786:   7%|▋         | 2/30 [00:05<01:19,  2.84s/it]

...

[I 2025-06-03 23:13:17,170] Trial 27 finished with value: 0.2990285147679009 and parameters: {'n_estimators': 450, 'learning_rate': 0.15000000000000002, 'max_depth': 7, 'num_leaves': 30, 'subsample': 1.0, 'colsample_bytree': 0.7999999999999999, 'reg_alpha': 0.6000000000000001, 'reg_lambda': 0.1}. Best is trial 17 with value: 0.30268358779591087.


Best trial: 17. Best value: 0.302684:  97%|█████████▋| 29/30 [03:18<00:07,  7.66s/it]

[I 2025-06-03 23:13:27,408] Trial 28 finished with value: 0.2901742433113319 and parameters: {'n_estimators': 450, 'learning_rate': 0.1, 'max_depth': 6, 'num_leaves': 30, 'subsample': 1.0, 'colsample_bytree': 0.7999999999999999, 'reg_alpha': 0.4, 'reg_lambda': 0.4}. Best is trial 17 with value: 0.30268358779591087.


Best trial: 17. Best value: 0.302684: 100%|██████████| 30/30 [03:28<00:00,  6.94s/it]


[I 2025-06-03 23:13:37,452] Trial 29 finished with value: 0.2826956990303734 and parameters: {'n_estimators': 450, 'learning_rate': 0.2, 'max_depth': 7, 'num_leaves': 20, 'subsample': 1.0, 'colsample_bytree': 0.7999999999999999, 'reg_alpha': 0.1, 'reg_lambda': 0.7000000000000001}. Best is trial 17 with value: 0.30268358779591087.

🎉 전체 학습 완료! 소요시간: 629.39초
🎯 최적화 완료! 결과를 확인하세요.


### 검증 데이터 

In [41]:
# 최적화된 LightGBM 코드와 호환되는 검증 코드

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import numpy as np

def validate_optimized_models(X_selected, train_df, targets_binary, 
                            binary_models_optimized, multi_model, 
                            test_size=0.2, random_state=42):
    """
    최적화된 모델들의 성능을 검증하는 함수
    """
    print("🔍 최적화된 모델 성능 검증 시작...\n")
    
    # ===== 이진 분류 모델들 검증 =====
    binary_results = {}
    
    for col in targets_binary:
        print(f"📊 {col} 이진 분류 모델 검증")
        print("-" * 50)
        
        y = train_df[col]
        
        # 검증 데이터 분할
        X_train, X_val, y_train, y_val = train_test_split(
            X_selected, y, test_size=test_size, random_state=random_state, stratify=y
        )
        
        # 기존 최적화된 모델 사용 또는 새로 학습
        if col in binary_models_optimized:
            # 이미 최적화된 모델이 있으면 검증용 데이터로 재학습
            model = binary_models_optimized[col]
            # 모델의 파라미터를 가져와서 새로운 모델 생성
            params = model.get_params()
            eval_model = LGBMClassifier(**params)
            eval_model.fit(X_train, y_train)
        else:
            print(f"⚠️ {col}에 대한 최적화된 모델이 없습니다. 기본 모델로 학습합니다.")
            eval_model = LGBMClassifier(random_state=random_state)
            eval_model.fit(X_train, y_train)
        
        # 예측 및 평가
        y_pred = eval_model.predict(X_val)
        f1 = f1_score(y_val, y_pred)
        
        print(f"F1 Score: {f1:.4f}")
        print("\n분류 보고서:")
        print(classification_report(y_val, y_pred))
        print("\n혼동 행렬:")
        print(confusion_matrix(y_val, y_pred))
        print("\n" + "="*60 + "\n")
        
        binary_results[col] = {
            'f1_score': f1,
            'model': eval_model,
            'predictions': y_pred,
            'true_labels': y_val
        }
    
    # ===== 다중 분류 모델 (S1) 검증 =====
    print("📊 S1 다중 분류 모델 검증")
    print("-" * 50)
    
    y_multi = train_df['S1']
    
    # 검증 데이터 분할
    X_train_multi, X_val_multi, y_train_multi, y_val_multi = train_test_split(
        X_selected, y_multi, test_size=test_size, random_state=random_state, stratify=y_multi
    )
    
    # 최적화된 다중분류 모델 사용
    if multi_model is not None:
        # 기존 모델의 파라미터로 새 모델 생성
        params = multi_model.get_params()
        eval_model_multi = LGBMClassifier(**params)
        eval_model_multi.fit(X_train_multi, y_train_multi)
    else:
        print("⚠️ 최적화된 다중분류 모델이 없습니다. 기본 모델로 학습합니다.")
        eval_model_multi = LGBMClassifier(
            objective='multiclass',
            num_class=len(y_multi.unique()),
            random_state=random_state
        )
        eval_model_multi.fit(X_train_multi, y_train_multi)
    
    # 예측 및 평가
    y_pred_multi = eval_model_multi.predict(X_val_multi)
    f1_multi = f1_score(y_val_multi, y_pred_multi, average='macro')
    
    print(f"F1 Score (Macro): {f1_multi:.4f}")
    print("\n분류 보고서:")
    print(classification_report(y_val_multi, y_pred_multi))
    print("\n혼동 행렬:")
    print(confusion_matrix(y_val_multi, y_pred_multi))
    
    multi_results = {
        'f1_score_macro': f1_multi,
        'model': eval_model_multi,
        'predictions': y_pred_multi,
        'true_labels': y_val_multi
    }
    
    return binary_results, multi_results

def compare_with_baseline(X_selected, train_df, targets_binary):
    """
    최적화된 모델과 기본 모델 성능 비교
    """
    print("🆚 기본 모델과 성능 비교")
    print("="*60)
    
    comparison_results = {}
    
    # 이진 분류 비교
    for col in targets_binary:
        y = train_df[col]
        X_train, X_val, y_train, y_val = train_test_split(
            X_selected, y, test_size=0.2, random_state=42, stratify=y
        )
        
        # 기본 모델
        baseline_model = LGBMClassifier(random_state=42)
        baseline_model.fit(X_train, y_train)
        baseline_pred = baseline_model.predict(X_val)
        baseline_f1 = f1_score(y_val, baseline_pred)
        
        comparison_results[col] = {
            'baseline_f1': baseline_f1
        }
        
        print(f"{col}: 기본 모델 F1 = {baseline_f1:.4f}")
    
    # 다중 분류 비교
    y_multi = train_df['S1']
    X_train, X_val, y_train, y_val = train_test_split(
        X_selected, y_multi, test_size=0.2, random_state=42, stratify=y_multi
    )
    
    baseline_multi = LGBMClassifier(
        objective='multiclass',
        num_class=len(y_multi.unique()),
        random_state=42
    )
    baseline_multi.fit(X_train, y_train)
    baseline_pred_multi = baseline_multi.predict(X_val)
    baseline_f1_multi = f1_score(y_val, baseline_pred_multi, average='macro')
    
    comparison_results['S1'] = {
        'baseline_f1': baseline_f1_multi
    }
    
    print(f"S1: 기본 모델 F1 (Macro) = {baseline_f1_multi:.4f}")
    
    return comparison_results

# 메인 실행 함수
def run_validation_suite(X_selected, train_df, test_X_selected, targets_binary):
    """
    전체 검증 스위트 실행
    """
    print("🚀 전체 모델 검증 시작")
    print("="*60)
    
    # 1. 최적화된 모델 학습 (기존 코드 사용)
    try:
        results = run_optimized_training(X_selected, train_df, test_X_selected, targets_binary)
        binary_preds_optimized, binary_models_optimized, multiclass_pred_optimized, multi_model = results
        print("✅ 최적화된 모델 학습 완료")
    except Exception as e:
        print(f"❌ 최적화된 모델 학습 실패: {e}")
        return None
    
    # 2. 성능 검증
    try:
        binary_results, multi_results = validate_optimized_models(
            X_selected, train_df, targets_binary, 
            binary_models_optimized, multi_model
        )
        print("✅ 모델 검증 완료")
    except Exception as e:
        print(f"❌ 모델 검증 실패: {e}")
        return None
    
    # 3. 기본 모델과 비교
    try:
        comparison_results = compare_with_baseline(X_selected, train_df, targets_binary)
        print("✅ 기본 모델 비교 완료")
    except Exception as e:
        print(f"❌ 기본 모델 비교 실패: {e}")
        comparison_results = None
    
    # 4. 결과 요약
    print("\n📋 최종 결과 요약")
    print("="*60)
    
    print("\n🎯 이진 분류 모델 성능:")
    for col, result in binary_results.items():
        optimized_f1 = result['f1_score']
        if comparison_results and col in comparison_results:
            baseline_f1 = comparison_results[col]['baseline_f1']
            improvement = ((optimized_f1 - baseline_f1) / baseline_f1) * 100
            print(f"  {col}: {optimized_f1:.4f} (기본 대비 {improvement:+.1f}% 개선)")
        else:
            print(f"  {col}: {optimized_f1:.4f}")
    
    print(f"\n🎯 다중 분류 모델 성능:")
    optimized_f1_multi = multi_results['f1_score_macro']
    if comparison_results and 'S1' in comparison_results:
        baseline_f1_multi = comparison_results['S1']['baseline_f1']
        improvement_multi = ((optimized_f1_multi - baseline_f1_multi) / baseline_f1_multi) * 100
        print(f"  S1: {optimized_f1_multi:.4f} (기본 대비 {improvement_multi:+.1f}% 개선)")
    else:
        print(f"  S1: {optimized_f1_multi:.4f}")
    
    return {
        'binary_results': binary_results,
        'multi_results': multi_results,
        'comparison_results': comparison_results,
        'optimized_models': binary_models_optimized,
        'multi_model': multi_model
    }


In [42]:

# 전체 검증 실행
validation_results = run_validation_suite(
    X_selected, train_df, test_X_selected, targets_binary
)

if validation_results:
    print("\n🎉 모든 검증 완료!")
    # 이진 분류 결과
    for col, result in validation_results['binary_results'].items():
        print(f"{col} F1 Score: {result['f1_score']:.4f}")
    
    # 다중 분류 결과
    multi_f1 = validation_results['multi_results']['f1_score_macro']
    print(f"S1 F1 Score (Macro): {multi_f1:.4f}")

[I 2025-06-03 23:13:49,923] A new study created in memory with name: no-name-e161f455-1b26-4957-92de-c3e7ed2d1634


🚀 전체 모델 검증 시작
🚀 GPU 최적화된 LightGBM 학습 시작
✅ GPU 사용 가능 확인됨

📊 Q1 최적화 중... (1/5)


Best trial: 0. Best value: 0.369582:   3%|▎         | 1/30 [00:03<01:37,  3.35s/it]

[I 2025-06-03 23:13:53,272] Trial 0 finished with value: 0.3695823045838303 and parameters: {'n_estimators': 250, 'learning_rate': 0.2, 'max_depth': 8, 'num_leaves': 70, 'subsample': 0.7, 'colsample_bytree': 0.7, 'reg_alpha': 0.0, 'reg_lambda': 0.9}. Best is trial 0 with value: 0.3695823045838303.


Best trial: 1. Best value: 0.391786:   7%|▋         | 2/30 [00:05<01:17,  2.76s/it]

...

[I 2025-06-03 23:23:48,054] Trial 27 finished with value: 0.2990285147679009 and parameters: {'n_estimators': 450, 'learning_rate': 0.15000000000000002, 'max_depth': 7, 'num_leaves': 30, 'subsample': 1.0, 'colsample_bytree': 0.7999999999999999, 'reg_alpha': 0.6000000000000001, 'reg_lambda': 0.1}. Best is trial 17 with value: 0.30268358779591087.


Best trial: 17. Best value: 0.302684:  97%|█████████▋| 29/30 [03:14<00:07,  7.44s/it]

[I 2025-06-03 23:23:58,258] Trial 28 finished with value: 0.2901742433113319 and parameters: {'n_estimators': 450, 'learning_rate': 0.1, 'max_depth': 6, 'num_leaves': 30, 'subsample': 1.0, 'colsample_bytree': 0.7999999999999999, 'reg_alpha': 0.4, 'reg_lambda': 0.4}. Best is trial 17 with value: 0.30268358779591087.


Best trial: 17. Best value: 0.302684: 100%|██████████| 30/30 [03:24<00:00,  6.80s/it]


[I 2025-06-03 23:24:08,211] Trial 29 finished with value: 0.2826956990303734 and parameters: {'n_estimators': 450, 'learning_rate': 0.2, 'max_depth': 7, 'num_leaves': 20, 'subsample': 1.0, 'colsample_bytree': 0.7999999999999999, 'reg_alpha': 0.1, 'reg_lambda': 0.7000000000000001}. Best is trial 17 with value: 0.30268358779591087.

🎉 전체 학습 완료! 소요시간: 621.62초
✅ 최적화된 모델 학습 완료
🔍 최적화된 모델 성능 검증 시작...

📊 Q1 이진 분류 모델 검증
--------------------------------------------------
F1 Score: 0.5412

분류 보고서:
              precision    recall  f1-score   support

           0       0.56      0.62      0.59        45
           1       0.57      0.51      0.54        45

    accuracy                           0.57        90
   macro avg       0.57      0.57      0.57        90
weighted avg       0.57      0.57      0.57        90


혼동 행렬:
[[28 17]
 [22 23]]


📊 Q2 이진 분류 모델 검증
--------------------------------------------------
F1 Score: 0.6735

분류 보고서:
              precision    recall  f1-score   support

   

In [43]:
# sample 기반 제출 포맷 가져오기
submission_final = sample_submission[['subject_id', 'sleep_date', 'lifelog_date']].copy()

# lifelog_date 기준으로 string → date 형식 통일
submission_final['lifelog_date'] = pd.to_datetime(submission_final['lifelog_date']).dt.date

# ID 만들기 (submission에서 예측한 결과와 연결하기 위해)
submission_final['ID'] = submission_final['subject_id'] + '_' + submission_final['lifelog_date'].astype(str)

In [44]:
# [Optuna 결과 사용]

# 예측 결과 연결
assert len(submission_final) == len(multiclass_pred_optimized) # shape 체크

# 다중 분류 예측 붙이기 (최적화된 결과)
submission_final['S1'] = multiclass_pred_optimized

# 이진 분류 결과 붙이기 (최적화된 결과)
for col in ['Q1', 'Q2', 'Q3', 'S2', 'S3']:
    submission_final[col] = binary_preds_optimized[col].astype(int)

# 최종 제출 형식 정렬
submission_final = submission_final[['subject_id', 'sleep_date', 'lifelog_date', 'Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']]

# 저장
submission_final.to_csv(submission_folder + submission_file, index=False)

print(f"✅ 최적화된 모델로 제출 파일이 저장되었습니다: {os.path.abspath(submission_file)}")

✅ 최적화된 모델로 제출 파일이 저장되었습니다: /home/user/miniconda3/envs/gpu_conda/src/lifelog-sleep-ictc-2025/minjeong/submission_final_mod5_0603_2259.csv


In [ ]:
# # ~ 그리드 서치 사용시 
# # 예측 결과 연결할 수 있도록 동일한 순서로 정렬
# # 보통 예측 결과는 test_df 기준이므로 정렬 보장되어야 함
# assert len(submission_final) == len(multiclass_pred_selected) # shape 체크

# # 다중 분류 예측 붙이기
# submission_final['S1'] = multiclass_pred_selected

# # 이진 분류 결과 붙이기
# for col in ['Q1', 'Q2', 'Q3', 'S2', 'S3']:
#     submission_final[col] = binary_preds_selected[col].astype(int) # 확률 아닌 class 예측


In [96]:

# 최종 제출 형식 정렬
submission_final = submission_final[['subject_id', 'sleep_date', 'lifelog_date', 'Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']]

# 저장
submission_final.to_csv(submission_folder + submission_file, index=False)

# VSCode에서는 files.download()가 작동하지 않으므로 대체

print(f"✅ 제출 파일이 저장되었습니다: {os.path.abspath(submission_file)}")

NameError: name 'submission_final' is not defined

In [97]:

# # 모델 저장 (추가 기능)
# import joblib

# # 모델 저장 디렉토리 생성
# os.makedirs('models', exist_ok=True)

# # 이진 분류 모델 저장
# for col in targets_binary:
#     joblib.dump(binary_preds[col], f'models/binary_model_{col}.pkl')
#     print(f"✅ 저장 완료: binary_model_{col}.pkl")

# # 다중 분류 모델 저장
# joblib.dump(model_s1, 'models/multiclass_model_S1.pkl')
# print(f"✅ 저장 완료: multiclass_model_S1.pkl")

In [ ]:
 
# # 결과 요약 - 특성 중요도 상위 20개 표시
# print("\n===== 주요 특성 중요도 (상위 20개) =====")
# print(feature_importance.head(20))

# # 모델 평가 (검증 데이터가 있는 경우)
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report, confusion_matrix

# # 검증용 데이터 분리
# X_train, X_val, y_train, y_val = train_test_split(
#     X, y_multi, test_size=0.2, random_state=42, stratify=y_multi
# )


===== 주요 특성 중요도 (상위 20개) =====
                   feature  importance
44         screen_on_ratio    0.021762
48             others_time    0.020832
46  screen_on_duration_avg    0.020828
7         activity_3_ratio    0.017365
62           wifi_rssi_max    0.016938
60          wifi_rssi_mean    0.016477
3    max_charging_duration    0.016160
26               rssi_mean    0.015765
0           charging_ratio    0.015758
8         activity_4_ratio    0.015606
41        light_night_mean    0.015580
47  screen_on_duration_max    0.015548
39               light_max    0.015141
2    avg_charging_duration    0.014409
31           altitude_mean    0.013876
38               light_std    0.013854
63       wifi_detected_cnt    0.013744
35             speed_max_x    0.013394
57                메시지_time    0.013303
19     Narration_monologue    0.013183
